# Module 2 Assignment

## Processing Data

In this section we load in the data and make it ready for NA and NLP.

### Importing Packages and Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Reading the two Scopus CSV data files 
scopus2022 = pd.read_csv('scopus 2022 2021.csv',  sep = ',')
scopus = pd.read_csv('scopus.csv',  sep = ',')

In [ ]:
#Appending the two data files 
data = scopus2022.append(scopus, ignore_index=True)

In [ ]:
#Getting an overview of the data
data.info()

In [ ]:
#Getting an overview of the data
data.columns

In [ ]:
#Specify data columns for Network Analysis
data_NA = data[['Authors', 'Author(s) ID', 'Title', 'Year', 'Cited by', 'References', 'Source title', 'Authors with affiliations']]

#Checking NA data
data_NA.head()

In [ ]:
#Checking NA data
data_NA.info()

In [ ]:
#Checking missing author ID
data_NA[data_NA['Author(s) ID'].isna()]

In [ ]:
#Dropping row with mising author ID
data_NA.dropna(subset=['Author(s) ID'], inplace=True)

In [ ]:
#Checking the authors that have been working on most titles. 
data_NA['Authors'].value_counts(ascending=False).nlargest(20)

In [ ]:
#Dropping row with "No author name available"
data_NA['Authors'] = data_NA['Authors'].replace('[No author name available]', ' ')
data_NA['Authors'] = data_NA['Authors'].replace(' ', np.nan)
data_NA.dropna(subset=['Authors'], inplace=True)

In [ ]:
#Checking titles with most authors
data_NA['Title'].value_counts(ascending=False).nlargest(20)

## Network Analysis

In this section we look into networks between authors. The aim is to identify co-authorships and locate important autors as well as detecting communities.

It is an undirecet network since we have authors working on papers together. Also, the nature of the network is bipartite as the nodes can be authors or papers. Hence, authors are connected if they have been working on the same paper. Furthermore, the network should not consist of self-loops, since the authors cannot co-author with themselves. 

### Importing packages and preparing data for Network Analysis

In [ ]:
#Filling in 'not specified' for authors with missing values in 'authors with affiliations'
data_NA['Authors with affiliations'] = data_NA['Authors with affiliations'].fillna('Not Specified')

In [ ]:
#Reset index after filling in 'authors with affiliations'
data_NA_Aff = data_NA["Authors with affiliations"].reset_index()

In [ ]:
#Split column 'Authors with affiliations' so that each author gets his/her own row
data_NA_Aff["Authors with affiliations"] = data_NA_Aff.loc[:,"Authors with affiliations"].apply(lambda x: x.split("; "))

In [ ]:
#Checking the data
data_NA_Aff.head()

In [ ]:
#Saving the new rows
data_NA_Aff = data_NA_Aff.explode(["Authors with affiliations"], ignore_index=True)
data_NA_Aff

In [ ]:
#Making new column for author name from the Authors with affiliations column
data_NA_Aff['Authors'] = data_NA_Aff.loc[:,"Authors with affiliations"].apply(lambda x: x.split(",")[0:2])

In [ ]:
data_NA_Aff

In [ ]:
#Making a new column for affiliations from the Authors with affiliations column
data_NA_Aff['Affiliations'] = data_NA_Aff.loc[:,"Authors with affiliations"].apply(lambda x: x.split(", ")[2:])

In [ ]:
#Checking the data
data_NA_Aff.head()

In [ ]:
#Change list to string for column Authors
f = lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
data_NA_Aff['Authors'] = data_NA_Aff['Authors'].apply(f)

In [ ]:
#Change list to string for column Affiliations
f = lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
data_NA_Aff['Affiliations'] = data_NA_Aff['Affiliations'].apply(f)

In [ ]:
#Drop duplicate authors to prepare for merge
data_NA_Aff['Authors'] = data_NA_Aff['Authors'].drop_duplicates()

In [ ]:
#Dropping empty values
data_NA_Aff = data_NA_Aff.dropna()

In [ ]:
#Reset index
data_NA_Aff = data_NA_Aff.reset_index()

In [ ]:
#Dropping columns
data_NA_Aff = data_NA_Aff.drop(columns=["level_0", "index"])

In [ ]:
#Remove comma to prepare for merge
data_NA_Aff['Authors'] = data_NA_Aff['Authors'].str.replace(',', '')

In [ ]:
#Moving on to data set data_NA - split the author column so that each author for each paper get his/her own row
data_NA["Authors"] = data_NA.loc[:,"Authors"].apply(lambda x: x.split(","))

In [ ]:
#Explode 
data_NA = data_NA.explode(["Authors"],ignore_index=True)
data_NA

In [ ]:
#Preparing column Authors for merge
for col in 'Authors':
    data_NA['Authors'] = data_NA['Authors'].apply(lambda x : x[1:] if x.startswith(" ") else x)

In [ ]:
#Adding a ID-number for each author 
data_NA['Author_ID'] = pd.factorize(data_NA['Authors'])[0]

In [ ]:
#Merging the data to get affiliation for each author 
data_NA_merge = pd.merge(data_NA, data_NA_Aff[['Authors', 'Affiliations']], on='Authors', how='left')

In [ ]:
#Checking the merged data
data_NA_merge

In [ ]:
data_NA_merge.info()

In [ ]:
#Fill in 'not specified when affiliations is empty
data_NA_merge['Affiliations'] = data_NA_merge['Affiliations'].fillna('Not Specified')

In [ ]:
#Checking the merged data 
data_NA_merge.info()

### Co-authorship network

Here we specify a network of co-authorships, specifying edges and nodes. The network shows which authors have been working on papers together. 

In [ ]:
from networkx.algorithms import bipartite
import networkx as nx

In [ ]:
#Importing packages for network analysis
import networkx as nx
import nxviz as nv
import matplotlib.pyplot as plt 

#Import the libraries and link to the bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show

from community import community_louvain 

In [ ]:
#Greating edges between authors working on the same title
data_NA_select = data_NA_merge[['Authors', 'Author_ID', 'Title', 'Affiliations', 'Year', 'Source title']]
edges = pd.merge(data_NA_select, data_NA_select, on='Title')

#Removing self-loops
edges = edges[edges.Author_ID_x != edges.Author_ID_y]

#Checking the edges
edges.head()

In [ ]:
#Grouping authors to aggregate muliple co-occurence and generating a weight. To see how often two autors have been working together
edges = edges.groupby(['Author_ID_x', 'Author_ID_y']).size().reset_index()

#Checking the edges
edges.head()

In [ ]:
edges.shape

In [ ]:
#Checking how many times an author has been working together with the same co-author
edges[0].value_counts()

In [ ]:
#Renaming the new column for weight
edges.rename({0:'weight'}, axis = 1, inplace=True)

In [ ]:
#Checking the number of titles an author has been working on
edges['Author_ID_x'].value_counts()


In [ ]:
#Making a list to keep authors that have been working on at least 3 papers
list_au_xy = edges['Author_ID_y'].value_counts().reset_index()[edges['Author_ID_y'].value_counts().reset_index()['Author_ID_y'] > 2]['index'].to_list()


In [ ]:
#Specifying a new edgelist 
edgesy = edges[edges['Author_ID_y'].isin(list_au_xy)]

In [ ]:
edgesy.shape

In [ ]:
edgesy.head(10)

In [ ]:
edgesxy = edgesy[edgesy['Author_ID_x'].isin(list_au_xy)]

In [ ]:
edgesxy.shape

In [ ]:
edgesxy.head(10)

In [ ]:
#Checking the number of edges
len(edges)

In [ ]:
# Create network object from pandas edgelist
G = nx.from_pandas_edgelist(edgesxy, source='Author_ID_x', target='Author_ID_y', edge_attr='weight', create_using=nx.Graph())

In [ ]:
#Nodes - Creating node-attribute dictionary 
node_attributes = data_NA_select[['Author_ID', 'Authors']].set_index('Author_ID').drop_duplicates().to_dict('index')

In [ ]:
nx.set_node_attributes(G, {G.degree(): 'degree'})

In [ ]:
nx.set_node_attributes(G, node_attributes)

In [ ]:
#Checking the number of nodes
len(G.nodes())

In [ ]:
#Checking the number of edges
len(G.edges())

In [ ]:
# G.degree()

In [ ]:
#Subset the graph keeping only nodes with degree > 1
G = nx.subgraph(G, [n for n,d in G.degree() if d > 1])

In [ ]:
len(G.nodes())

In [ ]:
len(G.edges())

#### Centrality

In [ ]:
#Calculate centralities
centrality_dgr = nx.degree_centrality(G)
centrality_eig = nx.eigenvector_centrality_numpy(G, weight = 'weight')
centrality_bet = nx.betweenness_centrality(G)


In [ ]:
# Setting centralities as as attributes of the Graph
nx.set_node_attributes(G, centrality_dgr, 'dgr')
nx.set_node_attributes(G, centrality_eig, 'eig')
nx.set_node_attributes(G, centrality_bet, 'bet')

In [ ]:
#Turn Graph object to DataFrame
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [ ]:
#Checking nodes_df
nodes_df.head()

##### Degree Centrality
Degree Centrality is a measure of how many ties a nodes has. So a author that have co-authored with more authors will have a higher centrality degree.

In [ ]:
#Closer look at degree centrality 
nodes_df.sort_values('dgr', ascending=False)[:10]

##### Eigenvector Centrality
Eigenvector Centrality is a measure of the influence a node has on a network - it takes into account the centrality of the autors connections.

In [ ]:
#Closer look at eigenvector centrality 
nodes_df.sort_values('eig', ascending=False)[:10]

##### Betweenness Centrality 
Betweenness Centrality is a measure of how often a author are in the shortes path between two other authors. It has to do with information passing between others. 

In [ ]:
#Closer look at betweennes centrality 
nodes_df.sort_values('bet', ascending=False)[:10]

#### Communities

In [ ]:
#Creating partitions
partition = community_louvain.best_partition(G)

In [ ]:
#Add to node attributes
nx.set_node_attributes(G, partition, 'partition')

In [ ]:
#Checking the dataframe after adding the partition column
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
nodes_df.head()

In [ ]:
#Checking the 20 largest communities
nodes_df.partition.value_counts()[:20]

In [ ]:
#Checking the number of partition (communities)
nodes_df.partition.nunique()

In [ ]:
#Taking out the people from the 10 largest communities to create a plot
top10_com = nodes_df.partition.value_counts()[:10].index 

#Creating nodes for 10 largest communities
top10_com_nodes = nodes_df[nodes_df['partition'].isin(top10_com)].index

#Create a subgraph 
g_sub = nx.subgraph(G, top10_com_nodes)

In [ ]:
#Creating a dataframe of the top 10 communities 
nodes_df_top10 = nodes_df[nodes_df['partition'].isin(top10_com)]

In [ ]:
nodes_df_top10

In [ ]:
#Looking at the 5 most important autors within the communities based on eigenvector centrality
top_authors = nodes_df_top10.groupby('partition')['eig'].nlargest(5).reset_index()

In [ ]:
top_authors

In [ ]:
#Adding back ID's and Names 
top_authors.rename({'level_1':'Author_ID'}, axis=1, inplace=True)
top_authors = pd.merge(top_authors, data_NA_select[['Authors','Author_ID']].drop_duplicates(), on='Author_ID', how='inner')

In [ ]:
top_authors

#### Visualisation of Co-author network

In [ ]:
#Saving a file to work on in Gephi
nx.write_gexf(G, "fintech.gexf")

##### Communities plot

In [ ]:
#Specifying central nodes for graph based on eigenvector 
top_central_nodes = nodes_df[nodes_df['eig'] > nodes_df['eig'].quantile(0.98)].index

In [ ]:
#TEST 
nodes_df_merge = pd.merge(nodes_df, data_NA_merge[['Authors', 'Year', 'Source title', 'Affiliations', 'Title']], on='Authors', how='left')

In [ ]:
#Creating a subset graph
g_sub = nx.subgraph(G, top_central_nodes)

In [ ]:
# Create and save a layout.
g_layout = nx.layout.spring_layout(g_sub) 
g_plot = hv.Graph.from_networkx(g_sub, g_layout).opts(tools=['hover'], node_color='partition')
labels = hv.Labels(g_plot.nodes, ['x', 'y'], 'Authors')

In [ ]:
# Import the libraries and link to the bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show
kwargs = dict(width=800, height=800, xaxis=None, yaxis=None)
opts.defaults(opts.Nodes(**kwargs), opts.Graph(**kwargs))

In [ ]:
# pip install scipy

In [ ]:
# make the plot
from holoviews.operation.datashader import datashade, bundle_graph
bundled = bundle_graph(g_plot)

In [ ]:
# show the plot
show(hv.render(bundled * labels.opts(text_font_size='6pt', text_color='white', bgcolor='gray')))

##### Community plot 

In [ ]:
# Setting the default figure size
defaults = dict(width=750, height=750, padding=0.1,
                xaxis=None, yaxis=None)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

In [ ]:
# Create and save layout for centrality plots
# G_layout = nx.layout.kamada_kawai_layout(G)

In [ ]:
# nx.set_node_attributes(G, partition, 'partition')

# g_plot_par = hv.Graph.from_networkx(G, G_layout).opts(tools=['hover'],
#                                                   node_color='partition', cmap=plt.cm.Set1,
#                                                   legend_position='right')

# show(hv.render(g_plot_par))